# **Train a model** - *Inspiré de la formation de Thibault Neveu*

# 0. Imports

In [1]:
import tensorflow as tf
import numpy as np

# 1. Be sure to use Tensor Flow 2.0

In [2]:
assert hasattr(tf, "function") # Be sure to use tensorflow 2.0

# 2. Start by creating a simple model

In [3]:
# Flatten
model = tf.keras.models.Sequential()
# Add layers
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

# 3. Setup the optimizer,  loss function and one metric

In [4]:
# The loss method
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
# The optimize
optimizer = tf.keras.optimizers.Adam()
# This metrics is used to track the progress of the training loss during the training
train_loss = tf.keras.metrics.Mean(name='train_loss')

# 4. Create a method which uses autograph to train the model

In [5]:
@tf.function
def train_step(image, targets):
    with tf.GradientTape() as tape:
        # Make a prediction
        predictions = model(image)
        # Get the error/loss using the loss_object previously defined
        loss = loss_object(targets, predictions)
    # Compute the gradient which respect to the loss
    gradients = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)

# 5. Train the model

In [6]:
for epoch in range(0, 10):
    for _ in range(0, 10):
        # Create fake inputs with two class
        # Create fake inputs
        inputs = np.zeros((2, 30))
        inputs[0] -= 1
        inputs[1] = 1
        # Create fake targets
        targets = np.zeros((2, 1))
        targets[0] = 0
        targets[1] = 1
        # Train the model
        train_step(inputs, targets)
    print("Loss: %s" % train_loss.result())
    train_loss.reset_state()

Loss: tf.Tensor(0.27524847, shape=(), dtype=float32)
Loss: tf.Tensor(0.0069277, shape=(), dtype=float32)
Loss: tf.Tensor(0.0010269617, shape=(), dtype=float32)
Loss: tf.Tensor(0.0003941603, shape=(), dtype=float32)
Loss: tf.Tensor(0.0002439168, shape=(), dtype=float32)
Loss: tf.Tensor(0.00018814045, shape=(), dtype=float32)
Loss: tf.Tensor(0.0001597324, shape=(), dtype=float32)
Loss: tf.Tensor(0.00014121635, shape=(), dtype=float32)
Loss: tf.Tensor(0.00012693123, shape=(), dtype=float32)
Loss: tf.Tensor(0.00011486292, shape=(), dtype=float32)


# 6. Important information
We used graph mode during training, which traditionally mean that model.predict could not be used, and the model had to be used in eager mode. However, due to recent updates in TensorFlow, it is now possible to use model.predict successfully.


In [7]:
try:
    input_ = np.zeros((1, 30)) + 1
    predictions = model.predict(input_)
    print("Prédiction réussie:", predictions)
except Exception as e:
    print("error ===>", e)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Prédiction réussie: [[1.397521e-04 9.998603e-01]]


However, we can still use the eager mode!

In [8]:
input_ = np.zeros((1, 30)) + 1
model(input_)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1.397521e-04, 9.998603e-01]], dtype=float32)>